![](https://i.ibb.co/G0sLBVs/30-Days-ML-Hero.png). 


Machine learning beginner → Kaggle competitor in 30 days. Non-coders welcome.



The dataset is used for this competition is synthetic, but based on a real dataset. The original dataset deals with predicting the amount of an insurance claim. Although the features are anonymized, they have properties relating to real-world features.

# Loading data

In [ ]:
from google.colab import files
files.upload()
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json
kaggle.json


In [ ]:
! kaggle competitions download -c 30-days-of-ml -p /content/drive/My\Drive/kaggle/30_ML

test.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
sample_submission.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
train.csv.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
%%capture
!pip install catboost

In [ ]:
%%capture
!pip install category_encoders

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, GridSearchCV, RandomizedSearchCV

import warnings

from sklearn.preprocessing import OrdinalEncoder 
from sklearn.preprocessing import LabelEncoder


from tensorflow import keras
from tensorflow.keras import layers, callbacks
from tensorflow.keras.callbacks import EarlyStopping
from keras import backend as K
from category_encoders import MEstimateEncoder

from sklearn.ensemble import VotingRegressor
from scipy.stats import reciprocal, uniform

from catboost import CatBoostRegressor
from catboost import Pool
import lightgbm as lgb
from lightgbm import LGBMRegressor


warnings.filterwarnings("ignore")
sns.set_style("darkgrid")
%matplotlib inline


/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
train = pd.read_csv('/content/drive/MyDrive/kaggle/30_ML/train.csv.zip')
test = pd.read_csv('/content/drive/MyDrive/kaggle/30_ML/test.csv.zip')

In [ ]:
train.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target
0,1,B,B,B,C,B,B,A,E,C,N,0.201470,-0.014822,0.669699,0.136278,0.610706,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850,8.113634
1,2,B,B,A,A,B,D,A,F,A,O,0.743068,0.367411,1.021605,0.365798,0.276853,0.533087,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083,8.481233
2,3,A,A,A,C,B,D,A,D,A,F,0.742708,0.310383,-0.012673,0.576957,0.285074,0.650609,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846,8.364351
3,4,B,B,A,C,B,D,A,E,C,K,0.429551,0.620998,0.577942,0.280610,0.284667,0.668980,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682,8.049253
4,6,A,A,A,C,B,D,A,E,A,N,1.058291,0.367492,-0.052389,0.232407,0.287595,0.686964,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823,7.972260


In [ ]:
test.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13
0,0,B,B,B,C,B,B,A,E,E,I,0.296227,0.686757,0.587731,0.392753,0.476739,0.376350,0.337884,0.321832,0.445212,0.290258,0.244476,0.087914,0.301831,0.845702
1,5,A,B,A,C,B,C,A,E,C,H,0.543707,0.364761,0.452967,0.929645,0.285509,0.860046,0.798712,0.835961,0.391657,0.288276,0.549568,0.905097,0.850684,0.693940
2,15,B,A,A,A,B,B,A,E,D,K,0.408961,0.296129,0.690999,0.740027,0.697272,0.683600,0.404089,0.879379,0.275549,0.427871,0.491667,0.384315,0.376689,0.508099
3,16,B,B,A,C,B,D,A,E,A,N,1.031239,0.356062,0.303651,0.895591,0.719306,0.777890,0.730954,0.644315,1.024017,0.391090,0.988340,0.411828,0.393585,0.461372
4,17,B,B,A,C,B,C,A,E,C,F,0.530447,0.729004,0.281723,0.444698,0.313032,0.431007,0.390992,0.408874,0.447887,0.390253,0.648932,0.385935,0.370401,0.900412


In [ ]:
def split_data(data, target_column):
    return data.drop(columns=[target_column], axis=1), data[target_column]

In [ ]:
X_test = test.drop(['id'], axis=1)

# Ensemble

In [ ]:
# X_encode, y_encode = split_data(train, 'target')

In [ ]:
# X_encode = X_encode.drop('id', axis=1)

In [ ]:
# X_en = X_encode.sample(frac=0.25)
# y_en = y_encode[X_en.index]
# X_pretrain = X_encode.drop(X_en.index)
# y_train = y_encode[X_pretrain.index]

In [ ]:
# encoder_1 = MEstimateEncoder(cols=["cat9"], m=4.0)
# encoder_1.fit(X_encode, y_encode)

# X_train = encoder_1.transform(X_pretrain)



In [ ]:
# features = X_train.copy()
# target = y_train
features, target = split_data(train, 'target')


In [ ]:
features.shape

(300000, 25)

In [ ]:
cat_feature = list(features.select_dtypes(include=['object']))
cat_feature 

['cat0',
 'cat1',
 'cat2',
 'cat3',
 'cat4',
 'cat5',
 'cat6',
 'cat7',
 'cat8',
 'cat9']

In [ ]:
# encoder = OrdinalEncoder()

encoder = LabelEncoder()

In [ ]:
# features[cat_feature] = encoder.fit_transform(features[cat_feature])
for cat in cat_feature:
    encoder = LabelEncoder()
    encoder.fit(list(features[cat].values)) 
    features[cat] = encoder.transform(list(features[cat].values))


In [ ]:
features.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13
0,1,1,1,1,2,1,1,0,4,2,13,0.201470,-0.014822,0.669699,0.136278,0.610706,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850
1,2,1,1,0,0,1,3,0,5,0,14,0.743068,0.367411,1.021605,0.365798,0.276853,0.533087,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083
2,3,0,0,0,2,1,3,0,3,0,5,0.742708,0.310383,-0.012673,0.576957,0.285074,0.650609,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846
3,4,1,1,0,2,1,3,0,4,2,10,0.429551,0.620998,0.577942,0.280610,0.284667,0.668980,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682
4,6,0,0,0,2,1,3,0,4,0,13,1.058291,0.367492,-0.052389,0.232407,0.287595,0.686964,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823


In [ ]:
features = features.drop('id', axis=1)

In [ ]:
test_id = test ['id']

In [ ]:
X_test = test.copy()
# X_test_cv_1 = X_test.copy()
# X_test_cv = encoder_1.transform(X_test_cv_1)
# X_test_cv[cat_feature] = encoder.transform(X_test_cv[cat_feature])
for cat in cat_feature:
    X_test[cat] = encoder.transform(list(X_test[cat].values))


X_test_cv = X_test.drop(['id'], axis=1)

In [ ]:
X_test_cv.head()

,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13
0,1,1,1,2,1,1,0,4,4,8,0.296227,0.686757,0.587731,0.392753,0.476739,0.376350,0.337884,0.321832,0.445212,0.290258,0.244476,0.087914,0.301831,0.845702
1,0,1,0,2,1,2,0,4,2,7,0.543707,0.364761,0.452967,0.929645,0.285509,0.860046,0.798712,0.835961,0.391657,0.288276,0.549568,0.905097,0.850684,0.693940
2,1,0,0,0,1,1,0,4,3,10,0.408961,0.296129,0.690999,0.740027,0.697272,0.683600,0.404089,0.879379,0.275549,0.427871,0.491667,0.384315,0.376689,0.508099
3,1,1,0,2,1,3,0,4,0,13,1.031239,0.356062,0.303651,0.895591,0.719306,0.777890,0.730954,0.644315,1.024017,0.391090,0.988340,0.411828,0.393585,0.461372
4,1,1,0,2,1,2,0,4,2,5,0.530447,0.729004,0.281723,0.444698,0.313032,0.431007,0.390992,0.408874,0.447887,0.390253,0.648932,0.385935,0.370401,0.900412


In [ ]:
#LGMB

params_lgb = {
    "task": "train",
    "boosting_type": "gbdt",
    "objective": "regression",
    "metric": "rmse",
    "learning_rate": 0.003899,
    'num_boost_round': 100000,
    "num_leaves": 62,
    "max_depth": 95,
    "feature_fraction": 0.2256,
    "bagging_fraction": 0.8805,
    "min_child_samples": 290,
    "reg_alpha": 9.562925,
    "reg_lambda": 9.35581,
    "max_bin": 881,
    "min_data_per_group": 125,
    "bagging_freq": 1,
    "cat_smooth": 96,
    "cat_l2": 19,
    "verbosity": -1,
    "bagging_seed": 42,
    "feature_fraction_seed": 42,
    "seed": 42,
    'random_state':42
}


#XGBboost

xgb_params = {
    'n_estimators': 5000,
    'learning_rate': 0.1235,
    'subsample': 0.95,
    'colsample_bytree': 0.11,
    'max_depth': 2,
    'booster': 'gbtree', 
    'reg_lambda': 66.1,
    'reg_alpha': 15.9,
    'random_state':42,
    'verbosity': 0
}

In [ ]:
XGB_ = XGBRegressor(**xgb_params)
LGBM = LGBMRegressor(**params_lgb)

In [ ]:
estimators = [XGB_, LGBM]

In [ ]:
X_train_XGB, X_val_XGB, y_train_XGB,y_val_XGB = train_test_split(features, target, test_size=0.25,
                                                                  random_state=42)

In [ ]:
features.shape, X_test_cv.shape

((300000, 24), (200000, 24))

In [ ]:
model = XGBRegressor(**xgb_params)
model.fit(X_train_XGB, y_train_XGB, eval_set = [(X_val_XGB, y_val_XGB)],
          early_stopping_rounds=200, verbose=200)

preds = model.predict(X_test_cv)
print(mean_squared_error(y_val_XGB, model.predict(X_val_XGB), squared=False))


[0]	validation_0-rmse:6.82735
Will train until validation_0-rmse hasn't improved in 200 rounds.
[200]	validation_0-rmse:0.730233
[400]	validation_0-rmse:0.725708
[600]	validation_0-rmse:0.723184
[800]	validation_0-rmse:0.721784
[1000]	validation_0-rmse:0.720679
[1200]	validation_0-rmse:0.719899
[1400]	validation_0-rmse:0.719098
[1600]	validation_0-rmse:0.718482
[1800]	validation_0-rmse:0.71794
[2000]	validation_0-rmse:0.717557
[2200]	validation_0-rmse:0.71723
[2400]	validation_0-rmse:0.716876
[2600]	validation_0-rmse:0.716574
[2800]	validation_0-rmse:0.71637
[3000]	validation_0-rmse:0.716221
[3200]	validation_0-rmse:0.716056
[3400]	validation_0-rmse:0.715954
[3600]	validation_0-rmse:0.715861
[3800]	validation_0-rmse:0.715852
[4000]	validation_0-rmse:0.715806
[4200]	validation_0-rmse:0.715749
[4400]	validation_0-rmse:0.715708
[4600]	validation_0-rmse:0.71567
[4800]	validation_0-rmse:0.715667
[4999]	validation_0-rmse:0.715623
0.7156304751640391


In [ ]:
sub_XG_full = pd.DataFrame()
sub_XG_full['Id'] = test_id
sub_XG_full['target'] = preds


sub_XG_full.to_csv('XGB_full-ensemble.csv', index=False)
files.download('XGB_full-ensemble.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
sub_XG_full.head()

,Id,target
0,0,8.114770
1,5,8.451990
2,15,8.433565
3,16,8.485857
4,17,8.129815


In [ ]:
preds = np.zeros(test.shape[0])
kf = KFold(n_splits=5, random_state=42, shuffle=True)
rmse=[]
idx=0


for train_idx, test_idx in kf.split(features, target):

    X_train_g, X_val_g = features.iloc[train_idx], features.iloc[test_idx]
    y_tr_g, y_val_g = target.iloc[train_idx], target.iloc[test_idx]


# XGBRegressor    

    model = XGBRegressor(**xgb_params)
    model.fit(X_train_g, y_tr_g, eval_set = [(X_val_g, y_val_g)],
              early_stopping_rounds=200, verbose=200)
             


    preds += model.predict(X_test_cv)/kf.n_splits

    rmse.append(mean_squared_error(y_val_g, model.predict(X_val_g), squared=False))
    print(idx+1,rmse[idx])
    idx +=1


[0]	validation_0-rmse:6.82899
Will train until validation_0-rmse hasn't improved in 200 rounds.
[200]	validation_0-rmse:0.730917
[400]	validation_0-rmse:0.726661
[600]	validation_0-rmse:0.724249
[800]	validation_0-rmse:0.722665
[1000]	validation_0-rmse:0.721542
[1200]	validation_0-rmse:0.720639
[1400]	validation_0-rmse:0.719825
[1600]	validation_0-rmse:0.719194
[1800]	validation_0-rmse:0.718644
[2000]	validation_0-rmse:0.718141
[2200]	validation_0-rmse:0.717751
[2400]	validation_0-rmse:0.71743
[2600]	validation_0-rmse:0.717137
[2800]	validation_0-rmse:0.716849
[3000]	validation_0-rmse:0.716724
[3200]	validation_0-rmse:0.716618
[3400]	validation_0-rmse:0.716467
[3600]	validation_0-rmse:0.716305
[3800]	validation_0-rmse:0.716239
[4000]	validation_0-rmse:0.716188
[4200]	validation_0-rmse:0.716174
[4400]	validation_0-rmse:0.716123
[4600]	validation_0-rmse:0.716072
[4800]	validation_0-rmse:0.71606
Stopping. Best iteration:
[4772]	validation_0-rmse:0.716041

1 0.7160439239219227
[0]	validati

In [ ]:
sub_XG = pd.DataFrame()
sub_XG['Id'] = test_id
sub_XG['target'] = preds

In [ ]:
sub_XG.head()

,Id,target
0,0,8.086397
1,5,8.407320
2,15,8.421388
3,16,8.506218
4,17,8.109425


In [ ]:
preds = np.zeros(test.shape[0])
kf = KFold(n_splits=5, random_state=42, shuffle=True)
rmse=[]
idx=0


for train_idx, test_idx in kf.split(features, target):

    X_train_g, X_val_g = features.iloc[train_idx], features.iloc[test_idx]
    y_tr_g, y_val_g = target.iloc[train_idx], target.iloc[test_idx]

# LGBMRegressor    

    model = LGBMRegressor(**params_lgb)
    model.fit(X_train_g, y_tr_g, eval_set = [(X_val_g, y_val_g)],
              early_stopping_rounds=200,verbose=False)
    preds += model.predict(X_test_cv)/kf.n_splits

    rmse.append(mean_squared_error(y_val_g, model.predict(X_val_g), squared=False))
    print(idx+1,rmse[idx])
    idx +=1

1 0.7166541135702886
2 0.7167587046434112
3 0.7185340457413194
4 0.7184551586950905
5 0.7169656524800676


In [ ]:
sub_LGBM = pd.DataFrame()
sub_LGBM['Id'] = test_id
sub_LGBM['target'] = preds

In [ ]:
sub_LGBM.head()

,Id,target
0,0,8.057325
1,5,8.317713
2,15,8.403558
3,16,8.475143
4,17,8.125825


#Развесовка

In [ ]:
!gdown --id 1xbEa--ywDx0ZubS8dayzFtb1_ztpVSNj
!gdown --id 1pwmoNVxZ9WjW71vdBj_Gt5ADZiFzRarE
!gdown --id 1vJJTxsbtIKt6FU8uj1bOOAlWxJs_QgFD

Downloading...
From: https://drive.google.com/uc?id=1xbEa--ywDx0ZubS8dayzFtb1_ztpVSNj
To: /content/XGB-ensembleKF.csv
4.92MB [00:00, 43.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1pwmoNVxZ9WjW71vdBj_Gt5ADZiFzRarE
To: /content/XGB_full-ensemble.csv
3.16MB [00:00, 49.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=1vJJTxsbtIKt6FU8uj1bOOAlWxJs_QgFD
To: /content/LGBM-ensemble.csv
4.92MB [00:00, 43.1MB/s]


In [ ]:
sub_XG = pd.read_csv('/content/XGB-ensembleKF.csv')

In [ ]:
sub_XG_full = pd.read_csv('/content/XGB_full-ensemble.csv')

In [ ]:
sub_LGBM = pd.read_csv('/content/LGBM-ensemble.csv')

In [ ]:
display(sub_LGBM.head().style.set_caption('LGBM - 0.71871'))
print()
print()
display(sub_XG.head().style.set_caption('XGB Kfold - 0.71769'))
print()
print()
display(sub_XG_full.head().style.set_caption('XGB - 0.71848'))

,Id,target
0,0,8.057325
1,5,8.317713
2,15,8.403558
3,16,8.475143
4,17,8.125825


,Id,target
0,0,8.086397
1,5,8.407320
2,15,8.421388
3,16,8.506218
4,17,8.109425


,Id,target
0,0,8.114770
1,5,8.451990
2,15,8.433565
3,16,8.485857
4,17,8.129815


In [ ]:
#XGB Kfold - 0.71769, LGBM - 0.71871, XGB - 0.71848
weights = [.90, .05, .05]

In [ ]:
sum(weights)

1.0

In [ ]:
sub_ense = pd.DataFrame()
sub_ense['Id'] = test['id']
sub_ense['target'] = (sub_XG['target'] * weights[0] + 
                      sub_LGBM['target'] * weights[1] +
                      sub_XG_full['target'] * weights[2]
                      )

In [ ]:
sub_ense.head()

,Id,target
0,0,8.086362
1,5,8.405074
2,15,8.421106
3,16,8.503646
4,17,8.111264


In [ ]:
sub_ense.to_csv('ensemble.csv', index=False)

In [ ]:
# sub_XG.to_csv('XGB-ensembleKF.csv', index=False)
sub_LGBM.to_csv('LGBM-ensemble.csv', index=False)
# sub_ense.to_csv('ensemble.csv', index=False)

In [ ]:
files.download('LGBM-ensemble.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
files.download('ensemble.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>